### Load libaries

In [1]:
import tensorflow as tf
import hls4ml
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import json
%matplotlib inline

### Load the data and transform

In [2]:
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, y = data['data'], data['target']
le = LabelEncoder()
y = le.fit_transform(y)
y = tf.keras.utils.to_categorical(y, 5)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

### Optional: train the model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.regularizers import l1
def three_layer_model(Inputs, nclasses, l1Reg=0, name='model_1'):
    x = Dense(64, activation='relu', kernel_initializer='lecun_uniform', 
              name='fc1_relu', kernel_regularizer=l1(l1Reg))(Inputs)
    x = Dense(32, activation='relu', kernel_initializer='lecun_uniform', 
              name='fc2_relu', kernel_regularizer=l1(l1Reg))(x)
    x = Dense(32, activation='relu', kernel_initializer='lecun_uniform', 
              name='fc3_relu', kernel_regularizer=l1(l1Reg))(x)
    predictions = Dense(nclasses, activation='softmax', kernel_initializer='lecun_uniform', 
                        name='output_softmax', kernel_regularizer=l1(l1Reg))(x)
    model = Model(inputs=Inputs, outputs=predictions, name=name)
    return model

In [ ]:
keras_model = three_layer_model(Input(shape=X_train_val.shape[1:], name='input_1'), y_train_val.shape[1])
keras_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['acc'])
keras_model.fit(X_train_val, y_train_val, 
                validation_split = 0.25, epochs = 20, 
                callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                              patience=5, 
                                                              restore_best_weights=True)])
keras_model.save('keras_3layer.h5',save_format='h5', overwrite=True)

### Load the model

In [ ]:
keras_model = tf.keras.models.load_model('keras_3layer.h5')
keras_model.summary()
y_keras = keras_model.predict(X_test)

### Create the hls4ml model and run C Simulation to get expected, quantized output

In [ ]:
hls_cfg = hls4ml.utils.config_from_keras_model(keras_model, granularity='name')
hls_model = hls4ml.converters.convert_from_keras_model(keras_model, output_dir='my-hls-test', hls_config=hls_cfg)
hls_model.compile()
y_hls4ml = hls_model.predict(X_test)

### Check difference between hls4ml and keras

In [ ]:
h, b = np.histogram((y_hls4ml - y_keras).flatten(), bins=np.linspace(-0.1,0.1,100))
plt.bar(b[:-1], h, width=b[1]-b[0], color='b', alpha=0.5, label='diff')
plt.legend()
plt.xlabel('y_hls4ml - y_keras')
plt.savefig('residual.png')

### Make ROC curves

In [ ]:
from sklearn.metrics import roc_curve
def roc(y_true, y, linestyle):
    for i in range(y_true.shape[1]):
        fpr, tpr, _ = roc_curve(y_true[:,i], y[:,i])
        plt.plot(tpr, fpr, linestyle=linestyle)
        
roc(y_test, y_keras, '-')
plt.gca().set_prop_cycle(None)
roc(y_test, y_hls4ml, '--')
plt.semilogy()
plt.xlabel("Signal Efficiency")
plt.ylabel("Background Efficiency")
plt.ylim(0.001,1)
plt.grid(True)

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--')]
plt.legend(lines, ['Keras', 'hls4ml'])
plt.savefig('ROC.png')

### Run Synthesis, read resources

In [ ]:
hls_model.build()
hls4ml.report.read_vivado_report('my-hls-test/')